In [5]:
using Oceananigans
using Oceananigans.Operators
using JLD2
using CairoMakie
using ImageFiltering: Kernel.gaussian, imfilter

In [19]:
#= 
dfm.jl
    Computes the down-front (y) mean and first-order correlations of the simulation output

    Arguments
        [1]: Folder containing simulation output, will also output to this folder
    Pass options in last argument
        i: use initialisation run
=#
ENV["JULIA_SCRATCH_TRACK_ACCESS"] = 0
include("../src-analysis/analysis_functions.jl")
include("../src-analysis/dfm_functions.jl")

foldername =  "../../scratch/Project/front-init-test-with-halos"
opts = "iv"
filename = 'i' ∈ opts ? "initialisation" : "output"

# Get grid and iterations
frames, ts = jldopen("$foldername/$filename.jld2") do file
    frames = parse.(Int, keys(file["timeseries/t"]))
    frames, [file["timeseries/t/$frame"] for frame in frames]
end
grid = jldopen("$foldername/$filename.jld2") do file
    file["serialized/grid"]
end

@info grid
# Get simulation parameters
sp = get_simulation_parameters(foldername)
@info sp
# input: u, v, w, b, ϕ
# Create a field for each by loading the first iteration in file

u_data = FieldTimeSeries("$foldername/$filename.jld2", "u"; iterations=frames[1:10])
v_data = FieldTimeSeries("$foldername/$filename.jld2", "v"; iterations=frames[1:10])
w_data = FieldTimeSeries("$foldername/$filename.jld2", "w"; iterations=frames[1:10])
b_data = FieldTimeSeries("$foldername/$filename.jld2", "b"; iterations=frames[1:10])
φ_data = FieldTimeSeries("$foldername/$filename.jld2", "φ"; iterations=frames[1:10])

u = FieldTimeSeries("$foldername/$filename.jld2", "u"; iterations=frames[51])[1]
v = FieldTimeSeries("$foldername/$filename.jld2", "v"; iterations=frames[51])[1]
w = FieldTimeSeries("$foldername/$filename.jld2", "w"; iterations=frames[51])[1]
b = FieldTimeSeries("$foldername/$filename.jld2", "b"; iterations=frames[51])[1]
φ = FieldTimeSeries("$foldername/$filename.jld2", "φ"; iterations=frames[51])[1]

input_fields = (; u, v, w, b, φ)
@info "Created input fields"
# ᶜⁿᶠ
# Mean fields
uᶠⁿᶜ = dfm(u)
vᶜⁿᶜ = dfm(v)
wᶜⁿᶠ = dfm(w)
bᶜⁿᶜ = dfm(b)
φᶜⁿᶜ = dfm(φ)

mean_fields = (; uᶠⁿᶜ, vᶜⁿᶜ, wᶜⁿᶠ, bᶜⁿᶜ, φᶜⁿᶜ)
@info "Created mean fields"
# Correlation fields
#  Self
u′u′ᶠⁿᶜ = dfm(KernelFunctionOperation{Face, Center, Center}(variance_kernel_func, grid, u, uᶠⁿᶜ))
v′v′ᶜⁿᶜ = dfm(KernelFunctionOperation{Center, Face, Center}(variance_kernel_func, grid, v, vᶜⁿᶜ))
w′w′ᶜⁿᶠ = dfm(KernelFunctionOperation{Center, Center, Face}(variance_kernel_func, grid, w, wᶜⁿᶠ))

#  Cross terms
u′v′ᶜⁿᶜ = dfm(KernelFunctionOperation{Center, Center, Center}(u′v′ᶜᶜᶜ_kernel_func, grid, u, uᶠⁿᶜ, v, vᶜⁿᶜ))
v′w′ᶜⁿᶜ = dfm(KernelFunctionOperation{Center, Center, Center}(v′w′ᶜᶜᶜ_kernel_func, grid, v, vᶜⁿᶜ, w, wᶜⁿᶠ))
w′u′ᶜⁿᶜ = dfm(KernelFunctionOperation{Center, Center, Center}(w′u′ᶜᶜᶜ_kernel_func, grid, w, wᶜⁿᶠ, u, uᶠⁿᶜ))

#  Buoyancy
u′b′ᶜⁿᶜ = dfm(KernelFunctionOperation{Center, Center, Center}(u′b′ᶜᶜᶜ_kernel_func, grid, u, uᶠⁿᶜ, b, bᶜⁿᶜ))
v′b′ᶜⁿᶜ = dfm(KernelFunctionOperation{Center, Center, Center}(v′b′ᶜᶜᶜ_kernel_func, grid, v, vᶜⁿᶜ, b, bᶜⁿᶜ))
w′b′ᶜⁿᶜ = dfm(KernelFunctionOperation{Center, Center, Center}(w′b′ᶜᶜᶜ_kernel_func, grid, w, wᶜⁿᶠ, b, bᶜⁿᶜ))

correlation_fields = (; u′u′ᶠⁿᶜ, v′v′ᶜⁿᶜ, w′w′ᶜⁿᶠ, u′v′ᶜⁿᶜ, v′w′ᶜⁿᶜ, w′u′ᶜⁿᶜ, u′b′ᶜⁿᶜ, v′b′ᶜⁿᶜ, w′b′ᶜⁿᶜ)
@info "Created correlation fields"
# Now we have defined all our operations, we loop over the frames

output_path = "$foldername/dfm2.jld2"
println()
for (i, frame) in enumerate(frames)
    # Get the data
    print("Calculating... $frame\r")
    @time update_fields!(input_fields, frame, "$foldername/$filename.jld2")
    # Compute 
    @time map(compute!, mean_fields)
    @time map(fill_halo_regions!, mean_fields)
    @time map(compute!, correlation_fields)
    @time map(fill_halo_regions!, correlation_fields)
    # Save to file
    write_output(mean_fields, correlation_fields, frame, output_path)
end
@info "Writing grid"
write_grid_times(grid, frames, ts, output_path)
@info "Finished!"


┌ Info: 1000×64×100 RectilinearGrid{Float64, Bounded, Periodic, Bounded} on CPU with 3×3×3 halo
│ ├── Bounded  x ∈ [-5000.0, 5000.0] regularly spaced with Δx=10.0
│ ├── Periodic y ∈ [-320.0, 320.0)   regularly spaced with Δy=10.0
└ └── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.0
[ Info: (f = 0.0001, H = 100, Nx = 1000, Ny = 64, Nz = 100, Ro = 0.6, Ri = 0, α = 0, Q = 10, c = 2, damping_frac = 0.5, turbulence_spinup = 100000, Lx = 10000.0, Ly = 640.0, ℓ = 1666.6666666666667, ΔN² = -2.4352353044352314e-5, N² = 0.0, N₀² = 2.4352353044352314e-5, B = 4.7231686290850545e-9, λ = 0.001, δ = 0.1)


[ Info: Created input fields
[ Info: Created mean fields
[ Info: Created correlation fields


  0.992229 seconds (18.28 k allocations: 286.564 MiB, 37.83% gc time, 30.10% compilation time)
  0.023653 seconds (1.22 k allocations: 343.234 KiB)
  0.000516 seconds (833 allocations: 124.031 KiB)
  1.523683 seconds (589.88 k allocations: 36.891 MiB, 59.58% compilation time: 100% of which was recompilation)
  0.001244 seconds (1.62 k allocations: 264.773 KiB)
  0.359137 seconds (3.95 k allocations: 285.680 MiB, 17.52% gc time)
  0.023369 seconds (1.22 k allocations: 343.195 KiB)
  0.000539 seconds (833 allocations: 124.000 KiB)
  0.608502 seconds (6.08 k allocations: 2.825 MiB)
  0.000793 seconds (1.62 k allocations: 264.648 KiB)
  0.333432 seconds (3.95 k allocations: 285.680 MiB, 8.95% gc time)
  0.022816 seconds (1.22 k allocations: 343.195 KiB)
  0.000489 seconds (833 allocations: 124.000 KiB)
  0.646413 seconds (6.08 k allocations: 2.825 MiB)
  0.000798 seconds (1.62 k allocations: 264.648 KiB)
  0.348018 seconds (3.95 k allocations: 285.680 MiB, 9.29% gc time)
  0.023720 seconds

[ Info: Writing grid


LoadError: UndefVarError: `t` not defined

In [28]:
Array(w′b′ᶜⁿᶜ.data[-2:end, 1, -2:end])

1006×106 Matrix{Float64}:
 0.0  0.0   0.0           0.0          …  0.0         0.0         0.0  0.0
 0.0  0.0   0.0           0.0             0.0         0.0         0.0  0.0
 0.0  0.0   0.0          -4.88241e-9      2.39223e-9  0.0         0.0  0.0
 0.0  0.0  -4.88241e-9   -4.88241e-9      2.39223e-9  2.39223e-9  0.0  0.0
 0.0  0.0   1.18649e-9    1.18649e-9      2.56172e-9  2.56172e-9  0.0  0.0
 0.0  0.0   1.81203e-10   1.81203e-10  …  1.40645e-9  1.40645e-9  0.0  0.0
 0.0  0.0   2.69035e-10   2.69035e-10     2.76289e-9  2.76289e-9  0.0  0.0
 0.0  0.0   2.29299e-11   2.29299e-11     2.09863e-9  2.09863e-9  0.0  0.0
 0.0  0.0   4.36459e-10   4.36459e-10     2.86569e-9  2.86569e-9  0.0  0.0
 0.0  0.0   2.65374e-10   2.65374e-10     2.40286e-9  2.40286e-9  0.0  0.0
 0.0  0.0   1.58955e-10   1.58955e-10  …  2.37219e-9  2.37219e-9  0.0  0.0
 0.0  0.0   4.47463e-10   4.47463e-10     2.50076e-9  2.50076e-9  0.0  0.0
 0.0  0.0  -1.66665e-9   -1.66665e-9      2.54541e-9  2.54541e-9  0.0  0.0